In [ ]:
import pandas as pd
from pandasql import sqldf
import boto3
import json
import os
# python -m spacy download en
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keras import models, layers
# from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import LabelBinarizer
# import numpy as np
from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

In [ ]:
with open('../boto-config.json') as json_data:
    boto_config = json.load(json_data)
    
s3 = boto3.client(
    's3',
    aws_access_key_id=boto_config['aws_access_key_id'],
    aws_secret_access_key=boto_config['aws_secret_access_key']
)

In [ ]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
src_file = data_dir + '/News_Category_Dataset.json'
src_file_cleaned = data_dir + '/News_Clean.csv'

# s3.upload_file(src_file, boto_config['buckets']['kaggle'], src_file)
# s3.upload_file(src_file_cleaned, boto_config['buckets']['kaggle'], src_file_cleaned)

# s3.download_file(boto_config['buckets']['kaggle'], src_file, src_file)
# s3.download_file(boto_config['buckets']['kaggle'], src_file_cleaned, src_file_cleaned)

In [ ]:
# data = pd.read_json(src_file, lines=True)
df = pd.read_csv(src_file_cleaned)
df = df[~df.clean_text.isnull()]

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data.isnull().values.any()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
# clean up text
nlp = spacy.load('en')

data['text'] = data['headline'] + ' ' + data['short_description']

docs = data['text'].tolist()

def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop | len(token.text) <= 3)

filtered_tokens = []
for doc in nlp.pipe(docs):
    tokens = [token.lemma_ for token in doc if token_filter(token) and token.text not in STOP_WORDS]
    filtered_tokens.append(' '.join(tokens))

data['clean_text'] = filtered_tokens

data.to_csv(src_file_cleaned, index=False)

In [ ]:
# tfidf with sklearn
tfidf = TfidfVectorizer()

bag = tfidf.fit_transform(df.loc[:, 'clean_text'])

In [ ]:
lbler = LabelEncoder()
labels = to_categorical(lbler.fit_transform(df['category']))
num_labels = labels.shape[1]

In [ ]:
train = tfidf.transform(df.loc[:, 'clean_text'])

In [ ]:
# train, test validation
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size = 0.2, random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 1)

In [ ]:
# nn with keras
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs = 10, batch_size=256, validation_data=(X_val, y_val))

In [ ]:
X_train